In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

/home/waccbip/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
train_data = pd.read_csv('002_train_padel_fps.csv')
test_data = pd.read_csv('002_test_padel_fps.csv')

print('traindata shape: ', train_data.shape)
print('testdata shape: ', test_data.shape)
train_data.head(5)

traindata shape:  (9347, 1445)
testdata shape:  (2334, 1445)


,nAcid,ALogP,ALogp2,AMR,apol,naAromAtom,nAromBond,nAtom,nHeavyAtom,nH,...,WTPT-1,WTPT-2,WTPT-3,WTPT-4,WTPT-5,WPATH,WPOL,XLogP,Zagreb,ACTIVITY
0,0,0.0435,0.001892,74.9344,39.292309,0,0,32,19,13,...,38.842627,2.044349,14.641057,5.661056,6.431126,726.0,26.0,2.168,100.0,1
1,0,1.6888,2.852045,139.5712,75.406204,0,0,63,35,28,...,73.193694,2.091248,23.704766,2.586746,12.936973,3948.0,57.0,5.582,198.0,1
2,0,1.3810,1.907161,100.6898,53.751446,0,0,47,25,22,...,49.885370,1.995415,20.501919,2.522207,17.979712,1794.0,35.0,2.103,120.0,0
3,0,2.5668,6.588462,117.7691,62.954204,0,0,55,27,28,...,54.636352,2.023569,12.038128,5.551697,6.486431,2074.0,41.0,3.090,138.0,1
4,0,1.7116,2.929575,107.6245,49.991516,0,0,37,25,12,...,51.678547,2.067142,16.921402,0.000000,14.395067,1320.0,47.0,2.420,138.0,1


In [3]:
continuous_cols = train_data.select_dtypes(include='float64').columns

Q1 = train_data[continuous_cols].quantile(0.25)
Q3 = train_data[continuous_cols].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

train_data[continuous_cols] = train_data[continuous_cols].clip(lower=lower_bound, upper=upper_bound, axis=1)
test_data[continuous_cols] = test_data[continuous_cols].clip(lower=lower_bound, upper=upper_bound, axis=1)

In [4]:
# from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import VarianceThreshold
from imblearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

def create_preprocessing_pipeline():
    pipeline = Pipeline([
        ('variance_selector', VarianceThreshold(threshold=0.01)),
        ('imputer', SimpleImputer()),
        ('scaler', RobustScaler())
    ])
    return pipeline

In [5]:
# from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import VarianceThreshold
from imblearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

def create_preprocessing_pipeline():
    pipeline = Pipeline([
        ('variance_selector', VarianceThreshold(threshold=0.01)),
        ('imputer', SimpleImputer()),
        ('scaler', RobustScaler())
    ])
    return pipeline

In [6]:
preprocessor = create_preprocessing_pipeline()

X_train = train_data.drop('ACTIVITY', axis=1)
y_train = train_data['ACTIVITY']

X_test = test_data.drop('ACTIVITY', axis=1)
y_test = test_data['ACTIVITY']


preprocessor.fit(X_train)
X_train = preprocessor.transform(X_train)
X_test = preprocessor.transform(X_test)

In [7]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0, class_weight='balanced')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)


In [8]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

print('accuracy: ', accuracy_score(y_test, y_pred))
print('f1_score: ', f1_score(y_test, y_pred))
print('precision_score: ', precision_score(y_test, y_pred))
print('recall_score: ', recall_score(y_test, y_pred))


accuracy:  0.7514995715509855
f1_score:  0.6674311926605505
precision_score:  0.7357774968394437
recall_score:  0.6107030430220357


In [11]:
'''
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

param_grid = {
    'n_estimators': [100, 200, 300],  # Number of trees in the forest
    'max_features': ['auto', 'sqrt', 'log2'],  # Number of features to consider at every split
    'max_depth': [None, 10, 20, 30],  # Maximum number of levels in tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split a node
    'min_samples_leaf': [1, 2, 4]  # Minimum number of samples required at each leaf node
}

grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=0, class_weight='balanced'),
                           param_grid=param_grid,
                           cv=3,  # Number of cross-validation folds (k)
                           verbose=2,  # Controls the verbosity: the higher, the more messages
                           n_jobs=-1,  # Number of jobs to run in parallel
                           scoring='accuracy')  # Can be 'f1', 'precision', 'recall', etc.
grid_search.fit(X_train, y_train)
'''

Fitting 3 folds for each of 324 candidates, totalling 972 fits
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.2s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.3s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   0.2s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=300; total time=   0.2s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=200; total time=   0.2s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=200; total time=   0.1s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=4, min_samples_spli

[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.4s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.3s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   0.2s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   0.2s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=200; total time=   0.1s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=100; total time=   0.2s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=100; total time=   0.2s
[CV] END max_depth=No

[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=   0.4s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   0.2s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   0.2s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=300; total time=   0.2s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=300; total time=   0.2s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=300; total time=   0.2s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=300; total time=   0.2s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=300; total time=   0.1s
[CV] END max_depth=No

[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=   0.4s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   0.2s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=300; total time=   0.2s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=300; total time=   0.3s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=300; total time=   0.2s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=300; total time=   0.1s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time=   0.2s
[CV] END max_depth=Non

[CV] END max_depth=20, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   0.0s
[CV] END max_depth=20, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   0.1s
[CV] END max_depth=20, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   0.1s
[CV] END max_depth=20, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=300; total time=   0.1s
[CV] END max_depth=20, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END max_depth=20, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END max_depth=20, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=200; total time=   0.1s
[CV] END max_depth=20, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=300; total time=   0.1s
[CV] END max_depth=20, max_features=

/home/waccbip/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
324 fits failed out of a total of 972.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
117 fits failed with the following error:
Traceback (most recent call last):
  File "/home/waccbip/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/waccbip/.local/lib/python3.10/site-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/home/waccbip/.local/lib/python3.10/site-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/home/waccbi

GridSearchCV(cv=3,
             estimator=RandomForestClassifier(class_weight='balanced',
                                              random_state=0),
             n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20, 30],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             scoring='accuracy', verbose=2)

[CV] END max_depth=20, max_features=log2, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=  18.1s
[CV] END max_depth=20, max_features=log2, min_samples_leaf=2, min_samples_split=2, n_estimators=300; total time=  51.2s
[CV] END max_depth=20, max_features=log2, min_samples_leaf=2, min_samples_split=5, n_estimators=300; total time=  55.5s
[CV] END max_depth=20, max_features=log2, min_samples_leaf=4, min_samples_split=2, n_estimators=100; total time=  18.0s
[CV] END max_depth=20, max_features=log2, min_samples_leaf=4, min_samples_split=2, n_estimators=300; total time=  53.0s
[CV] END max_depth=20, max_features=log2, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=  16.2s
[CV] END max_depth=20, max_features=log2, min_samples_leaf=4, min_samples_split=10, n_estimators=200; total time=  35.4s
[CV] END max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time= 1.9min
[CV] END max_depth=30, max_features=sq

[CV] END max_depth=20, max_features=log2, min_samples_leaf=4, min_samples_split=2, n_estimators=200; total time=  35.7s
[CV] END max_depth=20, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=  34.9s
[CV] END max_depth=20, max_features=log2, min_samples_leaf=4, min_samples_split=10, n_estimators=200; total time=  34.1s
[CV] END max_depth=30, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.1s
[CV] END max_depth=30, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=   0.1s
[CV] END max_depth=30, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.2s
[CV] END max_depth=30, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.1s
[CV] END max_depth=30, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   0.1s
[CV] END max_depth=30, max_features=aut

[CV] END max_depth=30, max_features=log2, min_samples_leaf=2, min_samples_split=5, n_estimators=300; total time=  40.0s
[CV] END max_depth=30, max_features=log2, min_samples_leaf=4, min_samples_split=2, n_estimators=100; total time=  13.3s
[CV] END max_depth=30, max_features=log2, min_samples_leaf=4, min_samples_split=2, n_estimators=200; total time=  25.4s
[CV] END max_depth=30, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=  25.2s
[CV] END max_depth=30, max_features=log2, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=  12.4s
[CV] END max_depth=30, max_features=log2, min_samples_leaf=4, min_samples_split=10, n_estimators=300; total time=  31.8s
[CV] END max_depth=20, max_features=log2, min_samples_leaf=2, min_samples_split=5, n_estimators=100; total time=  18.3s
[CV] END max_depth=20, max_features=log2, min_samples_leaf=2, min_samples_split=5, n_estimators=300; total time=  56.8s
[CV] END max_depth=20, max_features=lo

In [9]:
print("Best parameters found: ", grid_search.best_params_)
best_clf = grid_search.best_estimator_

# Predict on the test data using the best model
y_pred_best = best_clf.predict(X_test)

NameError: name 'grid_search' is not defined

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, balanced_accuracy_score
from sklearn.metrics import confusion_matrix

# Existing scores
print('Accuracy:', accuracy_score(y_test, y_pred_best))
print('F1 Score:', f1_score(y_test, y_pred_best))
print('Precision:', precision_score(y_test, y_pred_best))
print('Recall:', recall_score(y_test, y_pred_best))

# Additional Scores
# ROC AUC Score
print('ROC AUC:', roc_auc_score(y_test, y_pred_best))

# Balanced Accuracy
print('Balanced Accuracy:', balanced_accuracy_score(y_test, y_pred_best))

# Specificity Calculation
cm = confusion_matrix(y_test, y_pred_best)
tn, fp, fn, tp = cm.ravel()
specificity = tn / (tn + fp)
print('Specificity:', specificity)


In [33]:
X_val = pd.read_csv('X_val.csv')
y_val = pd.read_csv('y_val.csv')

In [34]:
continuous_cols = X_val.select_dtypes(include='float64').columns

Q1 = X_val[continuous_cols].quantile(0.25)
Q3 = X_val[continuous_cols].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

X_val[continuous_cols] = X_val[continuous_cols].clip(lower=lower_bound, upper=upper_bound, axis=1)

In [35]:
# from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import VarianceThreshold
from imblearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

def create_preprocessing_pipeline():
    pipeline = Pipeline([
        ('variance_selector', VarianceThreshold(threshold=0.01)),
        ('imputer', SimpleImputer()),
        ('scaler', RobustScaler())
    ])
    return pipeline

In [36]:
preprocessor = create_preprocessing_pipeline()
preprocessor.fit(X_val)
X_val = preprocessor.transform(X_val)

In [37]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0, class_weight='balanced')
clf.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', random_state=0)

In [45]:
feature_names_train = [f'feature_{i}' for i in range(X_train.shape[1])]
feature_names_val = [f'feature_{i}' for i in range(X_val.shape[1])]

# Convert both X_train and X_val to DataFrames with their respective column names
X_train_df = pd.DataFrame(X_train, columns=feature_names_train)
X_val_df = pd.DataFrame(X_val, columns=feature_names_val)

# Align the columns: keep only the columns from X_train in X_val
common_columns = X_train_df.columns.intersection(X_val_df.columns)

# Now filter X_val to have only the common columns
X_val_aligned = X_val_df[common_columns]

In [46]:
y_pred = clf.predict(X_val_aligned)

/home/waccbip/.local/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [47]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

print('accuracy: ', accuracy_score(y_val, y_pred))
print('f1_score: ', f1_score(y_val, y_pred))
print('precision_score: ', precision_score(y_val, y_pred))
print('recall_score: ', recall_score(y_val, y_pred))


accuracy:  0.577433628318584
f1_score:  0.17316017316017315
precision_score:  0.6896551724137931
recall_score:  0.09900990099009901
